<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Colab_RDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🖥️ Colab RDP (for Windows Remote Desktop)

This notebook sets up a full remote desktop (XFCE) on your Colab instance and makes it accessible using the **native Windows Remote Desktop app** via `xrdp` and an `ngrok` tunnel.

## ⚠️ IMPORTANT: Temporary Storage
> This Colab instance is temporary. **Any files you save *inside* the RDP (on the Desktop, in Documents, etc.) will be DELETED when the machine disconnects.**
>
> To save your work permanently, check the **'mount_google_drive'** box before running. This will add your Google Drive folder to the RDP's file manager.

## 📝 Instructions
---
1.  **Add your ngrok Token:**
    * Click the **Key icon (🔑)** in the left sidebar to open **Secrets**.
    * Create a new secret with the name `NGROK`.
    * Paste your ngrok authtoken (from https://dashboard.ngrok.com/get-started/your-authtoken) into the **Value** field.
    * Make sure **'Notebook access'** is enabled for this secret.
2.  **Credentials are fixed:**
    * **User:** `colab`
    * **Password:** `colab-rdp`
3.  **Run the Cell:** Press the "Play" button.
4.  **Wait:** The setup will take 5-10 minutes.
5.  **Connect:**
    * The output will show an **RDP Address** (like `0.tcp.ngrok.io:12345`).
    * Open the **Remote Desktop Connection** app on your Windows PC.
    * Paste the RDP Address into the "Computer" field and click Connect.
    * When prompted, enter the username (`colab`) and password (`colab-rdp`).

## Credits & Resources
---
* **Colab by:** Q.SARDOR ([Telegram](https://t.me/qsardorblog) | [GitHub](https://github.com/qsardor/))
* **More Colab Projects:** [Explore on GitHub](https://github.com/qsardor/GoogleColabProjects)


In [ ]:
#@title ▶️ Launch Windows RDP (xrdp) Desktop
#@markdown ---
#@markdown ## 1. Credentials (REQUIRED)
#@markdown ⚠️ **Action Required:** Add your ngrok token to Colab **Secrets (🔑)** with the name `NGROK`.
#@markdown
#@markdown The RDP credentials are fixed:
#@markdown **User:** `colab`
#@markdown **Password:** `colab-rdp`
#@markdown
#@markdown ## 2. Optional: Google Drive
mount_google_drive = False #@param {type:"boolean"}
#@markdown Check this box to mount your Google Drive inside the RDP (at `/home/colab/GoogleDrive`) for permanent file storage.
#@markdown ---

import os
import subprocess
from time import sleep
import sys
from google.colab import userdata
from google.colab import drive as colab_drive

log_file = "/content/xrdp_setup_log.txt"
RDP_USER = "colab"
RDP_PASS = "colab-rdp"

# --- Utility Functions ---
def check_gpu():
    try:
        gpu_name = subprocess.check_output(['nvidia-smi', '--query-gpu=gpu_name', '--format=csv,noheader']).decode('utf-8').strip()
        return f"✅ GPU Detected: {gpu_name}"
    except Exception:
        return "⚠️ GPU not found. Proceeding with CPU."

def run_cmd(command, step_name=""):
    if step_name:
        print(f"  -> ⏳ {step_name}...")
    try:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate()
        with open(log_file, 'a') as f:
            f.write(f"--- {step_name} ---\n")
            f.write(stdout)
            f.write(stderr)
        if process.returncode != 0:
            print(f"  -> ⚠️ Warning in step: {step_name}. Check log for details.")
    except Exception as e:
        print(f"  -> ❌ ERROR in step: {step_name}. {e}")
        with open(log_file, 'a') as f:
            f.write(f"--- ERROR: {step_name} ---\n{str(e)}\n")

# --- 0. System Info ---
print(f"--- 💻 SYSTEM INFO ---")
print(check_gpu())
print("-----------------------")
print(f"All installation logs will be saved to: {log_file}\n")

# --- Get NGROK Token from Secrets ---
NGROK_TOKEN = userdata.get('NGROK', None)

# --- 1. Install Dependencies ---
print("⏳ [Step 1/5] Installing Desktop & RDP server...")
if NGROK_TOKEN is None or NGROK_TOKEN == "":
    print("❌ ERROR: ngrok token not found. Did you add it to Colab Secrets (🔑) with the name 'NGROK'?")
else:
    os.environ['DEBIAN_FRONTEND'] = 'noninteractive'
    run_cmd('apt update -y', 'Update apt')
    run_cmd('apt install -y xfce4 xfce4-goodies xrdp python3-pyngrok', 'Install Desktop/xrdp')
    print("✅ Desktop & RDP installed.")

    # --- 2. Create & Configure User ---
    print(f"\n⏳ [Step 2/5] Creating user '{RDP_USER}'...")
    run_cmd(f"useradd -m {RDP_USER}", "Create user")
    run_cmd(f"adduser {RDP_USER} sudo", "Add user to sudo")
    run_cmd(f"echo '{RDP_USER}:{RDP_PASS}' | sudo chpasswd", "Set password")
    run_cmd(f"sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd", "Set bash shell")
    # Configure xrdp to use xfce session
    run_cmd(f"echo 'xfce4-session' > /home/{RDP_USER}/.xsession", "Set .xsession")
    run_cmd(f"chown -R {RDP_USER}:{RDP_USER} /home/{RDP_USER}", "Fix permissions")
    print("✅ User created and configured.")

    # --- 3. Start RDP Server ---
    print("\n⏳ [Step 3/5] Starting RDP server...")
    run_cmd('service xrdp start', 'Start xrdp service')
    sleep(2)
    print("✅ RDP server started.")

    # --- 4. Mount Google Drive (Optional) ---
    print("\n⏳ [Step 4/5] Checking for Google Drive...")
    if mount_google_drive:
        try:
            print("  -> ⏳ Mounting Google Drive...")
            colab_drive.mount('/content/drive')
            # Create a symlink into the user's home for easy access
            run_cmd(f'ln -s /content/drive /home/{RDP_USER}/GoogleDrive', 'Create Drive shortcut')
            print(f"✅ Google Drive mounted to /home/{RDP_USER}/GoogleDrive")
        except Exception as e:
            print(f"❌ ERROR: Could not mount drive. {e}")
    else:
        print("⚪ Skipping Google Drive mount.")

    # --- 5. Start ngrok Tunnel ---
    print("\n⏳ [Step 5/5] Starting ngrok tunnel...")
    try:
        from pyngrok import ngrok
        ngrok.set_auth_token(NGROK_TOKEN)
        # Open a TCP tunnel to the xrdp port 3389
        public_url_obj = ngrok.connect(3389, 'tcp')
        rdp_address = public_url_obj.public_url.replace('tcp://', '')

        print("\n\n🎉🎉🎉 \n")
        print(f"✅ Windows RDP Desktop is ready!")
        print("Open your Windows 'Remote Desktop Connection' app and use:")
        print(f"➡️  RDP Address: {rdp_address}")
        print(f"➡️  User: {RDP_USER}")
        print(f"➡️  Password: {RDP_PASS}")

        # Keep the cell running
        sleep(43200)
    except Exception as e:
        print(f"❌ ERROR starting ngrok: {e}")
        print("Please check your ngrok token.")
